# **Credit Risk Analysis and Prediction**

Analisis dan prediksi risiko kredit untuk mengevaluasi kemungkinan gagal bayar pinjaman berdasarkan data historis pelanggan.

Step Analisis:
1. Data Loading and Exploration
2. Data Preprocessing
3. Model Training
4. Model Evaluation
5. Inference (Testing Predictions)"

## **Import Library**

Tahap yang dilakukan di sesi ini adalah:
- Mengimpor semua pustaka yang diperlukan untuk analisis data, pra-pemrosesan, pembelajaran mesin, dan evaluasi model. 
- Mengimpor pandas untuk manipulasi data, seperti membaca file CSV, mengelola DataFrame, dan operasi data lainnya. 
- Mengimpor numpy untuk operasi numerik, seperti perhitungan matematis, manipulasi array, dan fungsi statistik. 
- Mengimpor fungsi train_test_split untuk membagi dataset menjadi data pelatihan dan pengujian secara acak. 
- Mengimpor StandardScaler untuk menstandarisasi fitur numerik dan LabelEncoder untuk mengubah data kategorikal menjadi numerik. 
- Mengimpor RandomForestClassifier, algoritma pembelajaran mesin berbasis ensemble untuk klasifikasi yang kuat terhadap data tidak seimbang. 
- Mengimpor metrik evaluasi: accuracy_score untuk akurasi, classification_report untuk laporan metrik terperinci, dan confusion_matrix untuk analisis kesalahan prediksi. 
- Mengimpor modul warnings untuk mengontrol peringatan yang dihasilkan selama eksekusi kode. 
- Mengatur pengabaian semua peringatan untuk menjaga output konsol tetap bersih dan fokus pada hasil.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

## **Data Loading and Exploration**

Tahap yang dilakukan di sesi ini:
- Memuat dataset dari file CSV dan melakukan eksplorasi awal untuk memahami struktur dan isi data. 
- Membaca file `credit_risk_dataset.csv` ke dalam DataFrame pandas untuk analisis lebih lanjut. file ini berisi data pinjaman dengan kolom seperti usia, pendapatan, dan status pinjaman.
- Menilai dataset yang digunakan.

#### **Load the dataset**

In [2]:
df = pd.read_csv('credit_risk_dataset.csv')

#### **Display few rows**

In [3]:
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


#### **Data information**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


#### **Check Missing Values**

In [5]:
df.isnull().sum()

person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

## **Data Preprocessing**

Tahap yang dilakukan di sesi ini:
- Melakukan pra-pemrosesan data untuk membersihkan, menangani nilai hilang, menghapus outlier, dan mengubah format data agar sesuai untuk pelatihan model.
- Handle missing values
- Menangani nilai yang hilang (missing values) pada kolom tertentu untuk memastikan data lengkap sebelum pelatihan model.

#### **Handle missing values**

In [6]:
df['loan_int_rate'].fillna(df['loan_int_rate'].median(), inplace=True)
df['person_emp_length'].fillna(df['person_emp_length'].median(), inplace=True)

Mengisi nilai yang hilang pada kolom `loan_int_rate` (suku bunga pinjaman) dengan nilai median dari kolom tersebut untuk menghindari bias; parameter `inplace=True` memodifikasi DataFrame secara langsung. Mengisi nilai yang hilang pada kolom `person_emp_length` (lama bekerja) dengan nilai median untuk menjaga konsistensi data; median dipilih karena tahan terhadap outlier.

In [7]:
df.isnull().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

#### **Remove outliers**
Menghapus data yang dianggap tidak realistis (outlier) untuk meningkatkan kualitas data dan akurasi model.

In [8]:
df = df[(df['person_age'] <= 100) & (df['person_emp_length'] <= 50)]

Memfilter DataFrame untuk hanya menyertakan baris dengan usia pelanggan `person_age` kurang dari atau sama dengan 100 tahun dan lama bekerja `person_emp_length` kurang dari atau sama dengan 50 tahun, karena nilai di luar rentang ini dianggap tidak wajar.

#### **Encode categorical variables**

Mengubah kolom kategorikal (teks) menjadi format numerik agar dapat digunakan oleh algoritma pembelajaran mesin.

In [9]:
le = LabelEncoder()
categorical_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

Membuat objek LabelEncoder dari sklearn untuk mengkodekan variabel kategorikal menjadi nilai numerik (misalnya, `RENT` menjadi 0, `MORTGAGE` menjadi 1). Mendefinisikan daftar kolom kategorikal yang akan dikodekan, yaitu kepemilikan rumah, tujuan pinjaman, grade pinjaman, dan riwayat gagal bayar. Menggunakan `LabelEncoder` untuk mengkodekan nilai kategorikal pada kolom saat ini menjadi nilai numerik; `fit_transform` melatih encoder dan langsung menerapkannya.

#### **Define features and target**

Memisahkan fitur (variabel independen) dan target (variabel dependen) untuk persiapan pelatihan model.

In [10]:
X = df.drop('loan_status', axis=1)
y = df['loan_status']

Membuat variabel X yang berisi semua kolom kecuali `loan_status` (target); axis=1 menunjukkan bahwa kolom akan dihapus. Menetapkan kolom `loan_status` sebagai variabel target y, yang menunjukkan apakah pinjaman gagal bayar (1) atau lunas (0).

#### **Split the data**

Membagi dataset menjadi dua bagian: data pelatihan (training) dan data pengujian (testing) untuk mengevaluasi performa model.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Membagi data dengan rasio 80% pelatihan dan 20% pengujian; random_state=42 memastikan pembagian konsisten setiap kali kode dijalankan.

#### **Scale numerical features**

Menstandarisasi fitur numerik agar memiliki mean 0 dan varians 1, yang diperlukan untuk beberapa algoritma pembelajaran mesin.

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Membuat objek `StandardScaler` untuk menstandarisasi fitur numerik dengan mengurangkan mean dan membagi dengan standar deviasi. Melatih scaler pada data pelatihan (menghitung mean dan standar deviasi) dan langsung menstandarisasi `X_train`. Menerapkan standarisasi pada data pengujian menggunakan parameter (mean dan standar deviasi) dari data pelatihan untuk menghindari kebocoran data.

## **Model Training**

Tahap yang dilakukan di sesi ini:
- Melatih model pembelajaran mesin menggunakan data pelatihan yang telah diproses.
- Using RandomForestClassifier for its robustness and ability to handle imbalanced datasets
- Memilih RandomForestClassifier karena kemampuannya menangani dataset yang tidak seimbang dan memberikan hasil yang kuat melalui pendekatan ensemble.


In [13]:
# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Membuat model `RandomForestClassifier` dengan 100 pohon keputusan `n_estimators=100`; `random_state=42` memastikan hasil konsisten. Melatih model pada data pelatihan (X_train, y_train) untuk mempelajari pola yang membedakan pinjaman yang gagal bayar dan lunas.

#### **Make predictions on the test set**


Menggunakan model yang telah dilatih untuk memprediksi status pinjaman pada data pengujian.

In [14]:
y_pred = model.predict(X_test)

Memprediksi label (gagal bayar atau lunas) untuk data pengujian (X_test) menggunakan model yang telah dilatih.

## **Model Evaluation**

Tahap yang dilakukan di sesi ini adalah mengevaluasi performa model dengan metrik seperti akurasi, laporan klasifikasi, dan matriks kebingungan.

#### **Model accuracy**

Mencetak skor akurasi, yaitu persentase prediksi yang benar dibandingkan dengan label sebenarnya pada data pengujian.

In [15]:
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Accuracy Score: 0.9338449731389102


#### **Classification report**

Mencetak header untuk laporan klasifikasi yang akan menampilkan metrik seperti presisi, recall, dan F1-score.

In [16]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5099
           1       0.97      0.72      0.83      1416

    accuracy                           0.93      6515
   macro avg       0.95      0.86      0.89      6515
weighted avg       0.94      0.93      0.93      6515



Mencetak laporan klasifikasi yang merinci performa model untuk setiap kelas (gagal bayar dan lunas), termasuk presisi, recall, dan F1-score.

#### **Confusion Matrix**

Mencetak header untuk matriks kebingungan yang menunjukkan distribusi prediksi benar dan salah. Mencetak matriks kebingungan, yaitu tabel 2x2 yang menunjukkan jumlah true positive, true negative, false positive, dan false negative.

In [17]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[5063   36]
 [ 395 1021]]


## **Testing Predictions**

Tahap yang dilakukan di sesi ini adalah menguji model dengan data sampel baru untuk melihat bagaimana model memprediksi status pinjaman pada kasus tertentu.


#### **Create sample test cases**

Membuat data sampel baru yang mewakili pelanggan hipotetis untuk menguji kemampuan prediksi model, membuat DataFrame baru dengan data sampel yang mencakup tiga pelanggan hipotetis dengan atribut yang relevan.

In [18]:
sample_data = pd.DataFrame({
    'person_age': [25, 40, 30],
    'person_income': [50000, 120000, 80000],
    'person_home_ownership': ['RENT', 'MORTGAGE', 'OWN'],
    'person_emp_length': [5.0, 10.0, 3.0],
    'loan_intent': ['EDUCATION', 'DEBTCONSOLIDATION', 'MEDICAL'],
    'loan_grade': ['B', 'A', 'C'],
    'loan_amnt': [10000, 20000, 15000],
    'loan_int_rate': [10.99, 7.9, 13.49],
    'loan_percent_income': [0.2, 0.17, 0.19],
    'cb_person_default_on_file': ['N', 'Y', 'N'],
    'cb_person_cred_hist_length': [3, 15, 7]})

#### **Preprocess sample data**

Melakukan pra-pemrosesan pada data sampel agar sesuai dengan format yang digunakan saat pelatihan model. Mengkodekan kolom kategorikal pada data sampel menjadi nilai numerik menggunakan `LabelEncoder`; `fit_transform` digunakan untuk melatih dan menerapkan encoder pada data sampel.

In [19]:
for col in categorical_cols:
    sample_data[col] = le.fit_transform(sample_data[col])

#### **Scale sample data**

Menstandarisasi data sampel agar memiliki skala yang sama dengan data pelatihan. Menerapkan standarisasi pada data sampel menggunakan scaler yang telah dilatih pada data pelatihan untuk memastikan konsistensi.

In [20]:
sample_data_scaled = scaler.transform(sample_data)

#### **Make predictions**

Menggunakan model untuk memprediksi status pinjaman pada data sampel yang telah diproses. Memprediksi status pinjaman (gagal bayar atau lunas) untuk data sampel menggunakan model yang telah dilatih.

In [21]:
predictions = model.predict(sample_data_scaled)

#### **Display the results**

Menampilkan hasil prediksi untuk data sampel dalam format yang mudah dibaca.

In [22]:
print("Inference Results for Sample Data:")
for i, pred in enumerate(predictions):
    status = 'Gagal Bayar' if pred == 1 else 'Lunas'
    print(f"Sample {i+1}: Predicted Loan Status = {status}")

Inference Results for Sample Data:
Sample 1: Predicted Loan Status = Lunas
Sample 2: Predicted Loan Status = Lunas
Sample 3: Predicted Loan Status = Lunas
